بسم الله الرحمن الرحيم


On a réussi à prendre les livres de fiqh maliki, maintenant on va essayer de mettre l'auteur...

In [111]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id = 'products')
#s = s.find_all('div', class_ = 'products product_list row grid')
#lines = s.find('h3', class_ = 's_title_block flex_child')
#Affichage du contenu de la page
#print(lines)

#Extraire les liens
urls = []
for line in s.findAll('a'):
    urls.append(line.get('href'))#line.text.split(\'n') est une liste de hcaine de caractères, donc [0] est 
    #seulement le str.
    #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n

Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.

In [112]:
#Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
urls = list(np.unique(L_string))
#urls

On va supprimer les liens qui ne renvoient pas vers des livres : 
 - Ceux qui finissent pas '.asc' ou 'desc
 - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
 - les "javascript"

In [113]:
#Suppression des liens ne renvoyant pas vers des livres.
for link in urls:
    if link.endswith('asc') or link.endswith('desc'):
        urls.remove(link)
    if link.startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?'):
        urls.remove(link)
urls.remove('javascript:')
urls.remove('javascript:;')
urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')


ValueError: list.remove(x): x not in list

In [ ]:
urls

On va maintenant essayer de regrouper les données des liens dans un dataframe.

## Essai pour le premier lien

In [177]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))

In [182]:
L_string

[['', 'Titre', 'شرح بداية المجتهد ونهاية المقتصد وبهامشه السبيل المرشد', ''],
 ['', 'Auteur', 'د. عبد الله العبادي', ''],
 ['', 'Volumes', '4', ''],
 ['', 'Pages', '2328', ''],
 ['', 'Édition', 'دار السلام', ''],
 ['', 'Couverture', 'Rigide', ''],
 ['', 'Volumes', '4', ''],
 ['', 'Format', '17x24cm', ''],
 ['', 'Harakat', 'Pas ou peu (ou que sur le Matn)', '']]

On essaye d'unifier pour enlever les deux "" et afficher en colonnes:

In [212]:
X = []
Y = []
for element in L_string:
    
    templist = list(np.unique(element))
    templist.remove('')
    X.append(templist[0])
    Y.append(templist[1])

In [213]:
pd.DataFrame({'c': X, "d": Y})

,c,d
0,Titre,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...
1,Auteur,د. عبد الله العبادي
2,4,Volumes
3,2328,Pages
4,Édition,دار السلام
5,Couverture,Rigide
6,4,Volumes
7,17x24cm,Format
8,Harakat,Pas ou peu (ou que sur le Matn)


On remarque qu'il récupère pas les données dans l'ordre, on aimerait avoir une condition sur des nombres tq:

pas 0, pas 1, 2, 3, pas 4, pas 5, 6, 7, pas 8, pas 9, 10, 11, pas 12, 13 

on remarque que la condition qu'il faut pour les différencier : les nombres qui valent 0 ou 1 modulo 4. (8%4 = 0, 9%4 = 1...) mais ca marche pas pour les autres.

In [228]:
L_string.

[['', 'Titre', 'شرح بداية المجتهد ونهاية المقتصد وبهامشه السبيل المرشد', ''],
 ['', 'Auteur', 'د. عبد الله العبادي', ''],
 ['', 'Volumes', '4', ''],
 ['', 'Pages', '2328', ''],
 ['', 'Édition', 'دار السلام', ''],
 ['', 'Couverture', 'Rigide', ''],
 ['', 'Volumes', '4', ''],
 ['', 'Format', '17x24cm', ''],
 ['', 'Harakat', 'Pas ou peu (ou que sur le Matn)', '']]

In [236]:
X = []
Y = []
for i in range(len(L_string)):
    templist = list(np.unique(L_string[i]))
    templist.remove('')
    if i%4 == 0 or i%4 ==1:
        X.append(templist[0])
        Y.append(templist[1])
    else:
        X.append(templist[1])
        Y.append(templist[0])


In [241]:
df = pd.DataFrame({'c': X, "d": Y})

In [248]:
df.d[0]

'شرح بداية المجتهد ونهاية المقتصد وبهامشه السبيل المرشد'

In [167]:
pd.DataFrame({"caracteristiques":X, "test":Y})

,caracteristiques,test
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
5,None,None
6,None,None
7,None,None
8,None,None


In [165]:
str(list(s)[0]).split('<')

['',
 'dl class="data-sheet flex_container">\n',
 'dt class="name">Titre',
 '/dt>\n',
 'dd class="value flex_child">شرح بداية المجتهد ونهاية المقتصد وبهامشه السبيل المرشد',
 '/dd>\n',
 '/dl>']

In [ ]:
L_string

On a une liste avec les principales informations d'un livre, on va essayer d'enlever les moins importantes et de garder les plus importantes.

In [125]:
L_string = list(np.unique(L_string))
try:
    L_string.remove('')
except ValueError:

    X = []
    Y = []
    for i in range(len(L_string)):
        if i%2==0:
            X.append(L_string[i])
        else:
            Y.append(L_string[i])
    df = pd.DataFrame({'test':pd.Series(X), 'test2':pd.Series(Y)})

In [127]:
L_string

['17x24cm',
 '2328',
 '4',
 'Auteur',
 'Couverture',
 'Fiche technique',
 'Format',
 'Harakat',
 'Pages',
 'Pas ou peu (ou que sur le Matn)',
 'Rigide',
 'Titre',
 'Volumes',
 'Édition',
 'د. عبد الله العبادي',
 'دار السلام',
 'شرح بداية المجتهد ونهاية المقتصد وبهامشه السبيل المرشد']